<a href="https://colab.research.google.com/github/carlos-castellanos99/Data-Science-Program-TEC/blob/main/RetoEmpleados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
#Importamos las librerías necesarias
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [122]:
#Cargamos los archivos para trabajar con ellos
from google.colab import files
uploaded = files.upload()

Saving empleadosRETO.csv to empleadosRETO.csv


In [123]:
#Leemos el archivo CSV
EmpleadosAttrition =  pd.read_csv('/content/empleadosRETO.csv')

In [124]:
EmpleadosAttrition.columns

Index(['Age', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
       'EducationField', 'EmployeeCount', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'Gender', 'JobInvolvement', 'JobLevel',
       'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome',
       'NumCompaniesWorked', 'HiringDate', 'Over18', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StandardHours', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'Attrition'],
      dtype='object')

In [125]:
#Eliminado columnas de la tabla (Punto 3)
EmpleadosAttrition.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1, inplace=True)

In [126]:
#Extraemos el año de la columna HiringDate (Punto 4)
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].str.split(pat='/').str[2].astype(int)


In [127]:
#Calculamos los años que el empleado lleva en la compañía hasta el año 2018
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']

In [128]:
#Renombramos la variable DistanceFromHome a DistanceFromHome_km (Punto 5 a)
EmpleadosAttrition.rename(columns={'DistanceFromHome': 'DistanceFromHome_km'}, inplace = True)

In [129]:
#Creamos una nueva variable DistanceFromHome que sea entera, es decir, solo con números (Punto 5 b)
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.split().str[0].astype(int)


In [130]:
#Eliminado columnas de la tabla (Punto 6)
EmpleadosAttrition.drop(['Year', 'HiringDate', 'DistanceFromHome_km'], axis=1, inplace=True)

In [131]:
#Creamos datframe SueldoPromedioDepto y gurdamos en una variable el sueldo promedio (Punto 7)
SueldoPromedioDepto =  EmpleadosAttrition.groupby(['Department'])[['MonthlyIncome']].mean()
SueldoPromedioDepto.rename(columns={'MonthlyIncome': 'SueldoPromedio'}, inplace = True)
SueldoPromedioDepto

,SueldoPromedio
Department,
Human Resources,6239.888889
Research & Development,6804.149813
Sales,7188.250000


In [132]:
#Escalamos la variable MonthlyIncome (Punto 8)
escalador = preprocessing.MinMaxScaler()
frame_escalado = escalador.fit_transform(EmpleadosAttrition[['MonthlyIncome']])
EmpleadosAttrition['MonthlyIncome'] = pd.Series(frame_escalado[:,0])

In [133]:
#Conviertimos todas las variables categóricas a numéricas
BusinessTravel = pd.get_dummies(EmpleadosAttrition.BusinessTravel, prefix = 'BusinessTravel')
Department = pd.get_dummies(EmpleadosAttrition.Department, prefix = 'Department')
EducationField = pd.get_dummies(EmpleadosAttrition.EducationField, prefix = 'EducationField')
Gender = pd.get_dummies(EmpleadosAttrition.Gender, prefix = 'Gender')
JobRole = pd.get_dummies(EmpleadosAttrition.JobRole, prefix = 'JobRole')
MaritalStatus = pd.get_dummies(EmpleadosAttrition.MaritalStatus, prefix = 'MaritalStatus')
Attrition = pd.get_dummies(EmpleadosAttrition.Attrition, prefix = 'Attrition')

EmpleadosAttrition_concat = pd.concat([EmpleadosAttrition, BusinessTravel, Department, EducationField, Gender, JobRole, MaritalStatus, Attrition], axis=1)

#Eliminado columnas de la tabla (Punto 3)
EmpleadosAttrition_concat.drop(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Attrition'], axis=1, inplace=True)

In [134]:
#Seleccionamos solo aquellas variables que tengan una correlación mayor o igual a 0.1 (Punto 11)

# Calculamos la correlación de cada variable con respecto a 'Attrition_Yes'
corr = EmpleadosAttrition_concat.corrwith(EmpleadosAttrition_concat['Attrition_Yes'])

# Filtramos las variables con correlación absoluta mayor o igual a 0.1
variables_filtradas = corr[abs(corr) >= 0.1].index.tolist()

# Creamos un nuevo DataFrame 'EmpleadosAttritionFinal' con las variables seleccionadas
EmpleadosAttritionFinal = EmpleadosAttrition_concat[variables_filtradas]

<ipython-input-134-c94ddf051d8c>:4: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = EmpleadosAttrition_concat.corrwith(EmpleadosAttrition_concat['Attrition_Yes'])


In [135]:
#Creamos una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal
pca = PCA()
pca.fit(EmpleadosAttritionFinal)
EmpleadosAttritionPCA  = pca.transform(EmpleadosAttritionFinal)
print(pca.explained_variance_ratio_ * 100)


[6.34313815e+01 2.41478065e+01 7.88482012e+00 2.10245261e+00
 6.51681722e-01 6.47500576e-01 3.01493054e-01 2.84948423e-01
 1.47700606e-01 1.03564120e-01 6.65502604e-02 6.37857065e-02
 4.60323985e-02 4.09156955e-02 2.83700349e-02 2.62718509e-02
 2.21422271e-02 2.58258415e-03 3.66982547e-33]


In [136]:
#Agregamos el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza, al frame EmpleadosAttritionFinal

EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(C0 = EmpleadosAttritionPCA[:,0])
EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(C1 = EmpleadosAttritionPCA[:,1])

In [137]:
EmpleadosAttritionFinal.to_csv("EmpleadosAttritionFinal.csv", index = False)